In [1]:
import numpy as np
import scanpy as sc
import anndata
import scvi
from scib_metrics.benchmark import Benchmarker
import torch
from functions import *
import scib_metrics
%matplotlib inline

/local/data/scvi-tools/scvi/_settings.py:63: UserWarning: Since v1.0.0, scvi-tools no longer uses a random seed by default. Run `scvi.settings.seed = 0` to reproduce results from previous versions.
  self.seed = seed
/local/data/scvi-tools/scvi/_settings.py:70: UserWarning: Setting `dl_pin_memory_gpu_training` is deprecated in v1.0 and will be removed in v1.1. Please pass in `pin_memory` to the data loaders instead.
  self.dl_pin_memory_gpu_training = (
/local/data/anaconda3/envs/BT/lib/python3.11/site-packages/numba/np/ufunc/dufunc.py:84: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  dispatcher = jit(_target='npyufunc',
/local/data/anaconda3/envs/BT/lib/python3.11/site-pac

In [3]:
adata = anndata.read_h5ad(filename="data/adataLung4.h5ad")
adata

AnnData object with n_obs × n_vars = 32472 × 4000
    obs: 'dataset', 'location', 'nGene', 'nUMI', 'patientGroup', 'percent.mito', 'protocol', 'sanger_type', 'size_factors', 'sampling_method', 'batch', 'cell_type', 'donor'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection'
    uns: 'hvg', 'pca'
    obsm: 'Scanorama', 'Unintegrated', 'X_pca'
    varm: 'PCs'
    layers: 'counts'

In [6]:
scvi.model.SCVI.setup_anndata(adata, layer="counts")

INFO:jax._src.xla_bridge:Unable to initialize backend 'cuda': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': INVALID_ARGUMENT: TpuPlatform is not available.


In [11]:
model = scvi.model.SCVI(adata)

In [14]:
model.train(plan_kwargs={"max_kl_weight":10,"n_epochs_kl_warmup":200})

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 1/246:   0%|          | 0/246 [00:00<?, ?it/s]

Epoch 2/246:   0%|          | 1/246 [00:01<06:22,  1.56s/it, v_num=1, train_loss_step=979, train_loss_epoch=947]

In [16]:
vaeSD = scvi.model.SCVI(adata, prior_distribution="sdnormal")
vaeMG = scvi.model.SCVI(adata, prior_distribution="mixofgaus")
vaeVP = scvi.model.SCVI(adata, prior_distribution="vamp")
vaeNF = scvi.model.SCVI(adata, prior_distribution="normalflow")

In [24]:
logger = TensorBoardLogger(save_dir="lightning_logs",name="sd_kl_max_exp_10_400")
vaeSD.train(plan_kwargs={"max_kl_weight":10,"n_epochs_kl_warmup":400},check_val_every_n_epoch=5,max_epochs=400,logger=logger)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Missing logger folder: lightning_logs/sd_kl_max_exp_10_400
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 400/400: 100%|██████████| 400/400 [10:16<00:00,  1.47s/it, v_num=0, train_loss_step=850, train_loss_epoch=953]    

`Trainer.fit` stopped: `max_epochs=400` reached.


Epoch 400/400: 100%|██████████| 400/400 [10:16<00:00,  1.54s/it, v_num=0, train_loss_step=850, train_loss_epoch=953]


In [25]:
logger = TensorBoardLogger(save_dir="lightning_logs",name="mog_kl_max_exp_10_400")
vaeMG.train(plan_kwargs={"max_kl_weight":10,"n_epochs_kl_warmup":400},check_val_every_n_epoch=5,max_epochs=400,logger=logger)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: lightning_logs/mog_kl_max_exp_10_400
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 2/400:   0%|          | 1/400 [00:01<10:59,  1.65s/it, v_num=0, train_loss_step=834, train_loss_epoch=886]

Epoch 400/400: 100%|██████████| 400/400 [11:55<00:00,  1.72s/it, v_num=0, train_loss_step=900, train_loss_epoch=944]    

`Trainer.fit` stopped: `max_epochs=400` reached.


Epoch 400/400: 100%|██████████| 400/400 [11:55<00:00,  1.79s/it, v_num=0, train_loss_step=900, train_loss_epoch=944]


In [26]:
logger = TensorBoardLogger(save_dir="lightning_logs",name="vamp_kl_max_exp_10_400")
vaeVP.train(plan_kwargs={"max_kl_weight":10,"n_epochs_kl_warmup":400},check_val_every_n_epoch=5,max_epochs=400,logger=logger)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: lightning_logs/vamp_kl_max_exp_10_400
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 1/400:   0%|          | 0/400 [00:00<?, ?it/s]

Epoch 400/400: 100%|██████████| 400/400 [12:42<00:00,  1.94s/it, v_num=0, train_loss_step=924, train_loss_epoch=941]    

`Trainer.fit` stopped: `max_epochs=400` reached.


Epoch 400/400: 100%|██████████| 400/400 [12:42<00:00,  1.91s/it, v_num=0, train_loss_step=924, train_loss_epoch=941]


In [23]:
logger = TensorBoardLogger(save_dir="lightning_logs",name="flow_kl_max_exp_10")
vaeNF.train(plan_kwargs={"max_kl_weight":10,"n_epochs_kl_warmup":200},check_val_every_n_epoch=5,max_epochs=200,logger=logger)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: lightning_logs/flow_kl_max_exp_10
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 200/200: 100%|██████████| 200/200 [28:22<00:00,  8.75s/it, v_num=0, train_loss_step=937, train_loss_epoch=947]    

`Trainer.fit` stopped: `max_epochs=200` reached.


Epoch 200/200: 100%|██████████| 200/200 [28:22<00:00,  8.51s/it, v_num=0, train_loss_step=937, train_loss_epoch=947]
